In [2]:
import pandas as pd
import numpy as np

##  Sales volume data for the estimation of the power consumption by shs usage


In [346]:

shs_sales_volumes = pd.read_csv('daten/shs_sales_volumes.csv', comment='#')
# compute the average of the product categories 5 to 7
shs_sales_volumes['tot_5-7'] = shs_sales_volumes[['5', '6', '7']].sum(axis=1)

shs_power_categories = pd.read_csv('daten/shs_power_per_product_categories.csv', comment='#')

# compute the average power for each category
shs_power_categories['power_av'] = shs_power_categories[['power_low', 'power_high']].mean(axis=1)

# select only the average power for the product categories 5 to 7
cat_power_av_5_7 = shs_power_categories['power_av'].loc[shs_power_categories.category >= 5].values
# compute the average power per region (multiply the number units sold per product categories by the average power of the respective category)
power_av_5_7 = shs_sales_volumes[['5', '6', '7']].values * cat_power_av_5_7
# compute the average unit power per region (divide the sum over the categories by the total unit sold)
shs_sales_volumes['weighted_tot_5-7 [W]'] = power_av_5_7.sum(axis=1) / shs_sales_volumes['tot_5-7'].values

SHS_POWER_CATEGORIES = shs_power_categories.set_index('category')
SHS_SALES_VOLUMES = shs_sales_volumes.set_index('region')

/home/Pierre.Duc/.virtualenvs/ndc-map/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [345]:
SHS_SALES_VOLUMES

,1,2,3,4,5,6,7,tot_5-7,weighted_tot_5-7 [W]
region,,,,,,,,,
SSA,494463,764638,450167,46594,58248,99217,13500,170965,67.294212
South Africa,0,0,7733,0,0,15558,0,15558,75.000000
West Africa,102992,67953,67290,3020,6634,43111,0,49745,69.665595
East Africa,366670,427640,371513,43501,51388,53284,13260,117932,66.003120
Central Africa,11743,230467,4631,0,186,2504,0,2690,72.234201
South Asia,450657,810281,99745,17357,0,5432,0,5432,75.000000
Middle East & North Africa,11878,240786,0,0,0,0,0,0,NaN
East Asia & Pacific,25108,17885,126357,1172,2809,0,0,2809,35.000000
Latin America,19946,12934,20280,0,862,0,0,862,35.000000


In [ ]:
# Global parameters
# variable
tier
invest_shs
invest_mg
# fixed
weight_rise
weight_experts

## Raw data loading

In [312]:
df = pd.read_csv('daten/raw_data.csv')

In [313]:
df.head()

,country,country_iso,region,pop_grid_share,hh_av_size,pop_2017,pop_2030,dark_rate,electrification_rate,hh_grid_share,...,hh_yearly_electricity_consumption,gdp_per_capita,mobile_money_2014,mobile_money_2017,ease_doing_business_index,corruption_index,weak_grid_index,rise_grid,rise_mg,rise_shs
0,Angola,AGO,SSA,0.232536,6.42,29784190,44712282,0.54,0.46,1.0,...,2025,4.170,0.12,0.21,181.0,19,4.7,33.33,52.50,62.22
1,Burundi,BDI,SSA,0.969213,4.80,10864250,15798849,0.90,0.10,1.0,...,365,320.000,0.01,NaN,152.0,22,16.6,16.67,47.50,11.11
2,Benin,BEN,SSA,0.537083,5.00,11175690,15628437,0.64,0.36,1.0,...,365,830.000,0.02,0.18,158.0,39,4.9,33.33,68.33,35.56
3,Burkina Faso,BFA,SSA,0.248028,5.70,19193380,27382488,0.77,0.23,1.0,...,365,671.000,0.03,0.33,143.0,42,9.8,33.33,57.50,22.22
4,Bangladesh,BGD,DA,0.653792,4.50,164669750,185584811,0.26,0.74,1.0,...,880,1.517,0.03,0.21,174.0,28,64.5,33.33,74.17,80.00


In [336]:
ELECTRIFICATION_OPTIONS = ['grid', 'mg', 'shs']
BAU_SENARIO = 'bau'
SE4ALL_SHIFT_SENARIO = 'se4all_shift'
PROG_SENARIO = 'prog'
SENARII = [BAU_SENARIO, SE4ALL_SENARIO, PROG_SENARIO]

MIN_TIER_LEVEL = 3
MIN_RATED_CAPACITY = {3: 200, 4: 800, 5: 2000} # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {3: 365, 4: 1250, 5: 3000} # index is TIER level [kWh/a]
# Find names for function on TIER lines 6 and 7

MENTI = pd.DataFrame({'mg': [3, 13. / 6, 19. / 6, 3.25, 11. / 3],
                    'shs': [23. / 12, 4.5, 37. / 12, 17. / 6, 41. / 12],
                   'labels': ['high_gdp', 'high_mobile_money', 'high_ease_doing_business', 'low_corruption', 'high_grid_weakness']
                   })
MENTI = MENTI.set_index('labels')

In [337]:
from daten.data_preparation import map_tier_yearly_consumption

# compute the grid and mg yearly consumption adjusted for tier level
for opt in ['grid', 'mg']:
    df['hh_%s_tier_yearly_electricity_consumption' % opt] = np.vectorize(map_tier_yearly_consumption)(
        df.hh_yearly_electricity_consumption, 
        df['hh_%s_share' % opt]
    )

#shs_unit_av_capacity  # SHS average capacity [Wp per sold unit]
#=WENN(C3="SSA";$'SHS sales volume'.$V$12;WENN(C3="DA";$'SHS sales volume'.$V$17;WENN(C3="LA";$'SHS sales volume'.$V$20;0)))
df['shs_unit_av_capacity'] = df.region.map(lambda region: SHS_SALES_VOLUMES.loc[region]['weighted_tot_5-7 [W]'])

opt = 'grid'
#=WENN(AA3<=$Tiers.$C$5;$Tiers.$C$4;WENN(AA3<=$Tiers.$D$5;$Tiers.$D$4))/1000
for tier_level in [4, 5]:
    df.loc[df.hh_yearly_electricity_consumption <= MIN_ANNUAL_CONSUMPTION[tier_level], 'cap_sn2_%s_tier_up' % opt] = MIN_RATED_CAPACITY[tier_level] / 1000

opt = 'mg'
#=$'Data_+_GIS'.T3*$'Data_+_GIS'.AF3
df['cap_sn2_%s_tier_up' % opt] = df.cap_sn2_grid_tier_up * df.hh_mg_share

opt ='shs'
#=WENN(V3<=$'SHS sales volume'.$D$6;$'SHS sales volume'.$D$7;$'SHS sales volume'.$D$8)
def shs_av_power(power_cat):
    return shs_power_categories.loc[power_cat, 'power_av']
df['cap_sn2_%s_tier_up' % opt] = df.shs_unit_av_capacity.map(lambda shs_cap: shs_av_power(6) if shs_cap <= shs_av_power(5) else shs_av_power(7))

In [ ]:
#Fixed inputs (from a csv)
# country,country_iso,region,hh_av_size,pop_2017,pop_2030,dark_rate,hh_grid_share,hh_yearly_electricity_consumption,gdp_per_capita,mobile_money_2014,mobile_money_2017,ease_doing_business_index
#country
#country_iso
#region
#dark_rate
#electrification_rate
#pop_shs_share
#pop_mg_share
#pop_grid_share
#pop_2017
#pop_2030

#hh_grid_share
#hh_mg_share
#shs_unit_av_capacity
#hh_yearly_electricity_consumption
#iea_regional_electricity_coverage

    
hh_grid_tier_peak_demand  # Grid peak demand grid [kW/HH], interpolated per Tier
#=WENN(UND(AB3>=$Tiers.$C$5;AB3<$Tiers.$D$5);AB3*$Tiers.$D$6+$Tiers.$D$7;WENN(UND(AB3>=$Tiers.$B$5;AB3<$Tiers.$C$5);AB3*$Tiers.$C$6+$Tiers.$C$7))/1000
if hh_grid_tier_yearly_electricity_consumption >= MIN_ANNUAL_CONSUMPTION[4] and hh_grid_tier_yearly_electricity_consumption < MIN_ANNUAL_CONSUMPTION[5]:
    hh_grid_tier_yearly_electricity_consumption*$Tiers.$D$6+$Tiers.$D$7/1000
    #$Tiers.$D$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 5 and TIER 4
    #$Tiers.$D$7 = MIN_RATED_CAPACITY[4] - MIN_ANNUAL_CONSUMPTION[4] * $Tiers.$D$6
else:
    if hh_grid_tier_yearly_electricity_consumption>=MIN_ANNUAL_CONSUMPTION[3] and hh_grid_tier_yearly_electricity_consumption<MIN_ANNUAL_CONSUMPTION[4]:
    #$Tiers.$C$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 4 and TIER 3
    #$Tiers.$C$7 = MIN_RATED_CAPACITY[3] - MIN_ANNUAL_CONSUMPTION[3] * $Tiers.$C$6
        hh_grid_tier_yearly_electricity_consumption*$Tiers.$C$6+$Tiers.$C$7/1000

hh_mg_tier_peak_demand  # MG peak demand grid [kW/HH], interpolated per Tier
#=WENN(UND(AC3>=$Tiers.$C$5;AC3<$Tiers.$D$5);AC3*$Tiers.$D$6+$Tiers.$D$7;WENN(UND(AC3>=$Tiers.$B$5;AC3<$Tiers.$C$5);AC3*$Tiers.$C$6+$Tiers.$C$7))/1000
if hh_mg_tier_yearly_electricity_consumption>=MIN_ANNUAL_CONSUMPTION[4] and hh_mg_tier_yearly_electricity_consumption < MIN_ANNUAL_CONSUMPTION[5]:
    hh_mg_tier_yearly_electricity_consumption*$Tiers.$D$6+$Tiers.$D$7/1000
    #$Tiers.$D$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 5 and TIER 4
    #$Tiers.$D$7 = MIN_RATED_CAPACITY[4] - MIN_ANNUAL_CONSUMPTION[4] * $Tiers.$D$6
else:
    if hh_mg_tier_yearly_electricity_consumption>=MIN_ANNUAL_CONSUMPTION[3] and hh_mg_tier_yearly_electricity_consumption<MIN_ANNUAL_CONSUMPTION[4]:
    #$Tiers.$C$6 = Ratio of MIN_RATED_CAPACITY and MIN_ANNUAL_CONSUMPTION between TIER 4 and TIER 3
    #$Tiers.$C$7 = MIN_RATED_CAPACITY[3] - MIN_ANNUAL_CONSUMPTION[3] * $Tiers.$C$6
        hh_mg_tier_yearly_electricity_consumption*$Tiers.$C$6+$Tiers.$C$7/1000

In [316]:
#endogenous results
df['pop_rel_growth'] = df.pop_2030 / df.pop_2017
df['pop_dark_2017'] = df.pop_2017 * df.dark_rate
df['pop_newly_electrified_2030'] = df.pop_rel_growth * df.pop_dark_2017
df['pop_electrified_2017'] = df.electrification_rate * df.pop_2017

## BaU senario data

In [292]:
bau_data = pd.read_csv('daten/bau.csv', comment='#')
bau_data = bau_data.set_index('region')

df['iea_regional_electricity_coverage'] = df['region'].map(lambda region: bau_data.loc[region]['iea_regional_electricity_coverage'])
df['bau_pop_newly_electrified'] = df.iea_regional_electricity_coverage * df.pop_newly_electrified_2030

for opt in ELECTRIFICATION_OPTIONS:
    #assign the regional electricity share for each options
    df['bau_pop_%s_share' % opt] = df['region'].map(lambda region: bau_data.loc[region]['%s_share' % opt])

## se4all+SHIFT senario data

In [317]:
from daten.data_preparation import (
    map_weak_grid_class,
    map_corruption_class,
    map_ease_doing_business_class,
    map_gdp_class,
    map_mobile_money_class
)

df['weak_grid_class'] = df['weak_grid_index'].map(map_weak_grid_class, na_action='ignore')
df['corruption_class'] = df['corruption_index'].map(map_corruption_class, na_action='ignore')
df['ease_doing_business_class'] = df['ease_doing_business_index'].map(map_ease_doing_business_class, na_action='ignore')
df['gdp_class'] = df['gdp_per_capita'].map(map_gdp_class, na_action='ignore')
df['mobile_money'] = df['mobile_money_2017'].fillna(df['mobile_money_2014'])
df['mobile_money_class'] = df['mobile_money'].map(map_mobile_money_class, na_action='ignore').fillna(0)

In [318]:
# apply the shift drives
for opt in ['mg', 'shs']:
    df['shift_menti_%s' % opt] = df.gdp_class * MENTI[opt]['high_gdp'] \
    + df.mobile_money_class * MENTI[opt]['high_mobile_money']
    + df.ease_doing_business_class * MENTI[opt]['high_ease_doing_business']
    + df.corruption_class * MENTI[opt]['low_corruption']
    + df.weak_grid_class * MENTI[opt]['high_grid_weakness']

In [319]:
# for se4all+SHIFT

WEIGHT_GRID = 0.8  # $RT_shift_factors.$O$2
WEIGHT = 0.2  # $RT_shift_factors.$P$2
RISE_INDICES = ['rise_grid', 'rise_mg', 'rise_shs']
SHIFT_MENTI = ['shift_menti_mg', 'shift_menti_shs']

#SUMME(S22:W22) --> df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1)
#SUMME($S22:$U22) --> df.loc[:,RISE_INDICES].sum(axis=1)
#SUMME($V22:$W22) --> df.loc[:,SHIFT_MENTI].sum(axis=1)

df['pop_get_grid_2030'] = df.pop_grid_share * df.pop_newly_electrified_2030

# to normalize the senarii weigthed sum
weighted_norm = df.loc[:,RISE_INDICES].sum(axis=1) * WEIGHT_GRID + df.loc[:,SHIFT_MENTI].sum(axis=1) * WEIGHT

non_zero_indices = df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1) != 0

for col in ['shift_grid_share','shift_grid_to_mg_share', 'shift_grid_to_shs_share']:
    # if the sum of the RISE indices and shift MENTI is 0 the corresponding rows in the given columns are set to 0
    df.loc[df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1) == 0, col] = 0

# share of population which will be on the grid in the se4all+SHIFT senario
#=WENN(SUMME(S22:W22)=0;0;S22*$RT_shift_factors.$O$2/(SUMME($S22:$U22)*$RT_shift_factors.$O$2+SUMME($V22:$W22)*$RT_shift_factors.$P$2))
df.loc[non_zero_indices, 'shift_grid_share'] = df.rise_grid * WEIGHT_GRID / weighted_norm

# share of population which will have changed from grid to mg in the se4all+SHIFT senario
#=WENN(SUMME(S22:W22)=0;0;(T22*$RT_shift_factors.$O$2+V22*$RT_shift_factors.$P$2)/(SUMME($S22:$U22)*$RT_shift_factors.$O$2+SUMME($V22:$W22)*$RT_shift_factors.$P$2))
df.loc[non_zero_indices, 'shift_grid_to_mg_share'] = (df.rise_mg * WEIGHT_GRID + df.shift_menti_mg * WEIGHT ) / weighted_norm

# share of population which will have changed from grid to shs in the se4all+SHIFT senario
#=WENN(SUMME(S23:W23)=0;0;(U23*$RT_shift_factors.$O$2+W23*$RT_shift_factors.$P$2)/(SUMME($S23:$U23)*$RT_shift_factors.$O$2+SUMME($V23:$W23)*$RT_shift_factors.$P$2))
df.loc[non_zero_indices, 'shift_grid_to_shs_share'] = (df.rise_shs * WEIGHT_GRID + df.shift_menti_shs * WEIGHT ) / weighted_norm


#=WENN(Y24>=X24;$Y24*D24;0)
#D24 -->  people get grid until 2030 (=SVERWEIS(B4;$'Data_+_GIS'.$B$3:$V$54;7;0)*SVERWEIS(B4;$'Data_+_GIS'.$B$3:$V$54;13;0))
# if the predicted mg share is larger than the predicted grid share, the number of people predited to use mg in the se4all+SHIFT senario is returned
# otherwise it is set to 0
df.loc[df.shift_grid_to_mg_share >= df.shift_grid_share, 'shift_pop_grid_to_mg'] = df.shift_grid_to_mg_share * df.pop_get_grid_2030
df.loc[df.shift_grid_to_mg_share < df.shift_grid_share, 'shift_pop_grid_to_mg'] = 0

#=WENN(Z25>=X25;Z25*$D25;0)
# if the predicted shs share is larger than the predicted grid share, the number of people predited to use shs in the se4all+SHIFT senario is returned
# otherwise it is set to 0
df.loc[df.shift_grid_to_shs_share >= df.shift_grid_share, 'shift_pop_grid_to_shs'] = df.shift_grid_to_shs_share * df.pop_get_grid_2030
df.loc[df.shift_grid_to_shs_share < df.shift_grid_share, 'shift_pop_grid_to_shs'] = 0

In [320]:
extract_results_senario(df, SE4ALL_SHIFT_SENARIO)

KeyError: "None of [Index(['shift_pop_grid_to_mgshift_pop_grid_to_shs'], dtype='object')] are in the [columns]"

## Exogenous results for all senarii

In [308]:
# exogenous results for BaU and se4all+SHIFT

def extract_results_senario(input_df, senario, regions=None):
    
    df = input_df.copy()
    
    if region is None:
        regions=['SSA', 'DA', 'LA']

        
    if senario == BAU_SENARIO:
        for opt in ELECTRIFICATION_OPTIONS:
            # not valid for other senario than bau at the moment
            # create a columns with regional electrification option shares 
            df['temp_%s' % opt] = df['region'].replace(regions, bau_data.loc[regions]['%s_share' % opt].to_list())

            # predicted number of people getting access to electricity (regional detail level)
            df['pop_get_%s_2030' % opt] = df['%s__pop_newly_electrified'] * df['temp_%s' % opt]
    elif senario in [SE4ALL_SHIFT_SENARIO, PROG_SENARIO]:
        #SUMME(AA4:AB4) --> df.loc[:,['shift_pop_grid_to_mg' 'shift_pop_grid_to_shs']].sum(axis=1) 
        #grid =D4-SUMME(AA4:AB4)
        opt = 'grid'
        # predicted number of people getting access to electricity (regional detail level)
        df['pop_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] - df.loc[:,['shift_pop_grid_to_mg' 'shift_pop_grid_to_shs']].sum(axis=1) 

        #mg =E5+AA5
        opt = 'mg'
        # predicted number of people getting access to electricity (regional detail level)
        df['pop_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] + df['shift_pop_grid_to_%s' % opt]

        #shs =F6+AB6
        opt = 'shs'
        # predicted number of people getting access to electricity (regional detail level)
        df['pop_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] + df['shift_pop_grid_to_%s' % opt]
    else:
        raise(ValueError)

    for opt in ELECTRIFICATION_OPTIONS:
        # predicted number of household getting access to electricity (regional detail level)
        df['hh_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] / hh_av_size
        # predicted power (in kW) that the access to electricity will represent (regional detail level)
        # the analysis is based on the peak demand for the grid and mg senarii, and the average
        # power of solar panel for shs senario
        if opt in ('grid' , 'mg'):
            df['hh_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['hh_%s_tier_peak_demand' % opt]
            df['hh_cap_scn2_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['cap_sn2_%s_tier_up' % opt]
        else:
            df['hh_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['shs_unit_av_capacity'] / 1000
            df['hh_cap_scn2_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['cap_sn2_%s_tier_up' % opt] / 1000
            
        

In [ ]:
#=WENN(Q22<4,5;0;WENN(Q22<=9;0,5;WENN(Q22>9;1;"NA")))

def map_weak_grid_class(weak_grid_idx):
    """Assign an index value to differentiate weak grid"""
    answer = 1
    if weak_grid_idx <= 4.5:
        answer = 0.5
    if weak_grid_idx <= 9:
        answer = 0
    return answer

#=WENN(O25<26;0;WENN(O25<=33;0,5;WENN(O25>33;1;"NA")))

def map_corruption_class(corruption_idx):
    """Assign an index value to differentiate corruption"""
    answer = 1
    if corruption_idx <= 33:
        answer = 0.5
    if corruption_idx <= 26:
        answer = 0
    return answer

#=WENN(M22<131;0;WENN(M22<=164;0,5;WENN(M22>164;1;"NA")))

def map_ease_doing_business_class(business_ease):
    """Assign an index value to differentiate ease of doing business"""
    answer = 1
    if business_ease <= 164:
        answer = 0.5
    if business_ease <= 131:
        answer = 0
    return answer

#Se4All

def map_gdp_class(gdp_per_capita):
    """Assign an index value to differentiate gdp per capita"""
    answer = 1
    if gdp_per_capita < 1500:
        answer = 0.5
    if gdp_per_capita < 700:
        answer = 0
    return answer
    
#=WENN(K5<=0,12;0;WENN(K5<=0,21;0,5;1))

def map_mobile_money_class(mobile_money):
    """Assign an index value to differentiate mobile_money"""
    answer = 1
    if mobile_money <= 0.21:
        answer = 0.5
    if mobile_money <= 0.12:
        answer = 0
    return answer


In [ ]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                    'B': [5, -1, 7, -3, 9],
                   'C': [4, 7, 3, 9, 1],
                   'D': [1, 5, 3, 0, 1],
                   'E': [7, 7, 2, 9, 7],
                    'Z': ['a', 'a', 'b', 'c', 'c']})
WEIGHT_GRID = 0.8  # $RT_shift_factors.$O$2
WEIGHT = 0.2  # $RT_shift_factors.$P$2



RISE_INDICES = ['A', 'B']
SHIFT_MENTI = []
# norm = df.loc[:,RISE_INDICES].sum(axis=1) * WEIGHT_GRID + df.loc[:,SHIFT_MENTI].sum(axis=1) * WEIGHT
# df['E'] * WEIGHT_GRID / norm

entries = ['shift_grid_share','shift_grid_to_mg_share', 'shift_grid_to_shs_share']
# df.loc[df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1) == 0, 'shift_grid_share' ] = '0'
for entry in entries:
    df.loc[df.B >0, entry ] = '0'

df